In [1]:
import random
import sys
import random
import re
import csv
from collections import defaultdict
import sys
from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics
import gensim 
import operator
from gensim.models import Word2Vec
import gensim 
import operator
from gensim.models import Word2Vec
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import transformers
import shap
from sklearn.metrics import (precision_score,recall_score,f1_score)
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import catboost
from catboost import *
from sklearn import tree
from sklearn.model_selection import train_test_split
#import xgboost as xg 
from sklearn.ensemble import ExtraTreesClassifier
#from catboost import CatBoostRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xg 
from sklearn.ensemble import RandomForestClassifier
import statistics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (precision_score,recall_score,f1_score)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import random
import sys
import random
import re
from sklearn import tree
from collections import defaultdict
import sys
from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics
import gensim 
import operator
from gensim.models import Word2Vec
import gensim 
import operator
from gensim.models import Word2Vec
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import transformers
import shap
from sklearn.metrics import (precision_score,recall_score,f1_score)
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import catboost
from catboost import *

    

def data_processing():
                        #Pre-process the Reviews /Users/khanfarabi/OneDrive - The University of Memphis/Explain_MLN/Explanation_yelp/

                        ifile1 = open("full-meta-data.txt")
                        revid = 0
                        users = defaultdict(list)
                        for ln in ifile1:
                            parts = ln.strip().split("\t")
                            users[parts[0]].append(revid)
                            revid = revid + 1
                        ifile1.close()
                        #print(users)


                        H11=defaultdict(list)
                        #sys.exit()
                        userids = []
                        #print(users)
                        c =0
                        #Select reviewer subset based on tunable parameters (max-reviews and min-reviews limit,sampling ratio)
                        minreviews =5
                        maxreviews =30
                        samplingratio =0.65
                        for u in users:
                            if len(users[u])>minreviews and len(users[u])<maxreviews:        
                                if random.random() < samplingratio:
                                    userids.append(u)
                                    c= c + len(users[u])




                        ifile = open("reviewContent.txt",encoding='ISO-8859-1')
                        ifile1 = open("full-meta-data.txt")
                        s_words = []
                        sfile = open("Words.txt")
                        for ln in sfile:
                            s_words.append(ln.strip())
                        sfile.close()
                        stopwords = []
                        sfile = open("stopwords.txt")
                        for ln in sfile:
                            stopwords.append(ln.strip())
                        sfile.close()

                        flags = (re.UNICODE if sys.version < '3' and type(text) is unicode
                                 else 0)
                        ofile = open("all_revs1.txt",'w')
                        ofile1 = open("metadata.txt",'w')
                        cnt = 0
                        revid = 0
                        qrat={}
                        windex = defaultdict(list)
                        #Tunable parameter to keep non-sentiment words
                        PNonSentWords = 0.50
                        WORDS={}
                        w_per_ht=defaultdict(list)
                        w_per_user=defaultdict(list)
                        Rev_text_map={}
                        for ln in ifile:
                            ln1 = ifile1.readline()
                            parts1 = ln1.strip().split("\t")
                            #print(parts1)
                            if parts1[0] not in userids:
                                continue
                            #if cnt >= 10000:
                            #    break
                            keep = []
                            parts = ln.strip().split("\t")
                            for word in re.findall(r"\w[\w']*", parts[3], flags=flags):
                                if word.isdigit() or len(word)==1:
                                    continue
                                word_lower = word.lower()
                                if word_lower in stopwords:
                                    continue
                               # if word_lower in s_words:
                                    #keep.append(word_lower)
                                elif random.random() < PNonSentWords:
                                    if not any(c.isdigit() for c in word_lower) and "'" not in word_lower:
                                        keep.append(word_lower)
                            if float(parts1[2])<=2:
                                cl = 0
                            elif float(parts1[2])==3:
                                cl = 1
                            elif float(parts1[2])>=4:
                                cl = 2
                            if len(keep)>=25:
                                cnt = cnt + 1
                                ofile.write(" ".join(keep)+"\t"+str(cl)+"\n")
                                WORDS[revid]=keep

                                qrat[revid]=cl
                                Rev_text_map[revid]=parts[3]
                                H11[parts1[1]].append(revid)
                                ofile1.write(ln1)
                                for w in keep:
                                    windex[w].append(revid)
                                    w_per_ht[w].append(parts1[1])
                                    w_per_user[w].append(parts1[0])
                                revid = revid + 1
                        ofile.close()
                        ofile1.close()
                        ifile.close()
                        ifile1.close()
                        '''
                        #Tunable parameter (keep words only if repeated in > NumReps reviews)
                        NumReps = 10

                        #Filter review words
                        ifile = open("all_revs1.txt",encoding="ISO-8859-1")
                        ofile = open("processed_revs_1.txt",'w')
                        for ln in ifile:
                            parts = ln.strip().split("\t")
                            keep = []
                            for w in parts[0].split(" "):
                                if len(windex[w])>NumReps:
                                    keep.append(w)
                            ofile.write(" ".join(keep)+"\t"+parts[1]+"\n")
                        ofile.close()
                        ifile.close()
                        '''
                        print("Total Reviews="+str(len(WORDS)))
                        return s_words,stopwords,WORDS,qrat,H11,Rev_text_map






def Shap_Explanation_Engine(s_words,stopwords,WORDS,qrat,H11,Rev_text_map):

                                                    #Balancing Review Data for both positive and negative
                                                    WORDSt={}
                                                    t1=[]
                                                    t2=[]
                                                    t3=[]
                                                    c1=0
                                                    c2=0
                                                    c3=0
                                                    for y in WORDS:
                                                        if qrat[y]==0:
                                                            if c3<500:
                                                                t1.append(y)
                                                                WORDSt[y]=WORDS[y]
                                                                c3=c3+1
                                                        elif qrat[y]==1:
                                                            continue
                                                            #if c1<102:
                                                                #t2.append(y)
                                                                #WORDS1[y]=WORDS[y]
                                                                #c1=c1+1
                                                        elif qrat[y]==2:
                                                            if c2<500:
                                                                t3.append(y)
                                                                WORDSt[y]=WORDS[y]
                                                                c2=c2+1
                                                    print(len(t1),len(t2),len(t3),len(WORDSt))
                                                    for k in WORDSt:
                                                        if qrat[k]==1:
                                                            print(k)
                                                    d_tt={}
                                                    d_tt[0]='negative'
                                                    d_tt[2]='positive'
                                                    WORDS_u={}
                                                    aw={}
                                                    c0=0
                                                    c1=0
                                                    K=500
                                                    for t in WORDSt:
                                                        if qrat[t]==0:
                                                            if c0<K:
                                                                WORDS_u[t]=WORDSt[t]
                                                                aw[t]=WORDSt[t]
                                                                c0=c0+1
                                                        elif qrat[t]==2:
                                                            if c1<K:
                                                                WORDS_u[t]=WORDSt[t]
                                                                aw[t]=WORDSt[t]
                                                                c1=c1+1
                                                    print(len(WORDS_u),len(aw))
                                                    #annotation
                                                    WORDS_uf={}
                                                    ann={}
                                                    kk=1.0
                                                    for t in WORDS_u:
                                                        m=0
                                                        gg=[]
                                                        for cvv in WORDS_u[t]:
                                                            if cvv not in gg:
                                                                #if m/float(len(WORDS_u[t]))<=kk:
                                                                    gg.append(cvv)

                                                        WORDS_uf[t]=gg
                                                       # print(len(gg),len(WORDS_u[t])         

                                                    for k1 in WORDS_uf:
                                                        vcc=[]
                                                        c=0
                                                        for t2 in WORDS_uf[k1]:
                                                            if t2 in s_words:
                                                                #print(t2)
                                                                if t2 not in vcc:
                                                                    #if c<500:
                                                                        vcc.append(t2)
                                                                        c=c+1
                                                        if len(vcc)>0:
                                                            ann[k1]=vcc
                                                    #annotted word features
                                                    wf2=[]
                                                    for t in ann:
                                                        for j in ann[t]:
                                                            if j not in wf2:
                                                                wf2.append(j)

                                                    #train and target data

                                                    train_r=[]
                                                    targets_r=[]
                                                    m_tid_tr1={}
                                                    wr=[]
                                                    tr=[]
                                                    wr1=[]
                                                    tr1=[]
                                                    c=0
                                                    c1=0
                                                    tw_wm={}
                                                    for t in WORDS_u:
                                                        s=''
                                                        vb=[]
                                                        #if twit_count[t]==1:
                                                        for tt in WORDS_u[t]:
                                                                s=s+str(tt)+" "
                                                                train_r.append(tt)
                                                                targets_r.append(qrat[t])
                                                        vb.append(s)
                                                        wr.append(s)
                                                        wr1.append(vb)
                                                        tr.append(qrat[t])
                                                        tw_wm[t]=s
                                                    unique_words=[]
                                                    ss=set( train_r)
                                                    for w1 in ss:
                                                            if w1 not in unique_words:
                                                                unique_words.append(w1)
                                                    #Shap

                                                    #shap.initjs()
                                                    # Kernal Shap words_train targets


                                                    corpus_train, corpus_test, y_train, y_test = train_test_split(train_r,targets_r, test_size=0.5, random_state=7)
                                                    vectorizer = TfidfVectorizer(min_df=1)
                                                    X_train = vectorizer.fit_transform(corpus_train)
                                                    X_test = vectorizer.transform(corpus_test)
                                                    model =svm.LinearSVC(C=100) #sklearn.linear_model.LogisticRegression(penalty="l1", C=0.1)
                                                    #KNeighborsClassifier(n_neighbors=5)
                                                    #RandomForestClassifier(max_depth=2, random_state=1)
                                                    #svm.LinearSVC(C=100) #sklearn.linear_model.LogisticRegression(penalty="l1", C=0.1)
                                                    model.fit(X_train,y_train)
                                                    p = model.predict(X_test)
                                                    prr={}
                                                    for jj in range(0,len(corpus_test)):
                                                        prr[corpus_test[jj]]=int(p[jj])
                                                    print(f1_score(y_test,p,average='micro'))
                                                    explainer =shap.LinearExplainer(model, X_train, feature_dependence="independent")
                                                    shap_values = explainer.shap_values(X_test)
                                                    X_test_array = X_test.toarray() # we need to pass a dense version for the plotting functions
                                                    feature_names=vectorizer.get_feature_names()
                                                    print(len(feature_names),len(shap_values))
                                                    #shap.summary_plot(shap_values, X_test_array, feature_names=vectorizer.get_feature_names())
                                                    shape_w={}
                                                    fr={}
                                                    feature_sh_v=[]
                                                    for jj in range(0,len(corpus_train)):
                                                          if abs(sum(shap_values[jj]))>0.4:
                                                                                  m=abs(sum(shap_values[jj]))
                                                                                  if corpus_train[jj] not in fr:
                                                                                                  fr[corpus_train[jj]]=abs(sum(shap_values[jj]))
                                                                                  elif corpus_train[jj]  in fr:
                                                                                        if m>fr[corpus_train[jj]]:
                                                                                            fr[corpus_train[jj]]=m
                                                    dd1=sorted(fr.items(), key=operator.itemgetter(1),reverse=True)
                                                    for tt in dd1:
                                                           feature_sh_v.append(tt[0])
                                                    '''
                                                    feature_sh_v1=[]
                                                    for v in feature_sh_v:
                                                        n=v.split()
                                                        for k in n:
                                                            if k not in feature_sh_v1:
                                                                feature_sh_v1.append(k)
                                                    '''
                                                    #shap explanations

                                                    shap_exp={}
                                                    for t in WORDSt:
                                                        gh=[]
                                                        c=0
                                                        for k in WORDSt[t]:
                                                            if k in prr:
                                                                if qrat[t]==prr[k]:
                                                                    if k in feature_sh_v:
                                                                        if k not in gh:
                                                                            if c<5:
                                                                                gh.append(k)
                                                                                c=c+1
                                                        if len(gh)>0:
                                                                shap_exp[t]=gh

                                                    for tt in shap_exp:
                                                         pass#print(tt,shap_exp[tt])
                                                    shap_all={}
                                                    shap_all_p={}
                                                    shap_all_n={}

                                                    for t in shap_exp:
                                                        if t in ann:
                                                            c=0
                                                            vb=0
                                                            for zz in shap_exp[t]:
                                                                if zz in ann[t]:
                                                                    if vb<5:
                                                                            c=c+1
                                                                            vb=vb+1
                                                            if len(shap_exp[t])>0:
                                                                s=float(c)/len(shap_exp[t])
                                                                if s>0:
                                                                    shap_all[t]=s
                                                    ss=0
                                                    for k in shap_all:
                                                        ss=ss+float(shap_all[k])

                                                    acc=ss/len(shap_all)
                                                    print("Shap Accuracy without human-feedback"+"\n")
                                                    print("Shap Accuracy")
                                                    print(acc)
                                                    print("Shap only positive reviews")
                                                    for t in shap_exp:
                                                        if t in ann:
                                                            if qrat[t]==2:
                                                                        c=0
                                                                        vb1=0
                                                                        for zz in shap_exp[t]:
                                                                            if zz in ann[t]:
                                                                                if vb1<5:
                                                                                    c=c+1
                                                                                    vb1=vb1+1
                                                                        if len(shap_exp[t])>0:
                                                                            s=float(c)/len(shap_exp[t])
                                                                            if s>0:
                                                                                shap_all_p[t]=s
                                                    ss1=0
                                                    for k in shap_all_p:
                                                        ss1=ss1+float(shap_all_p[k])

                                                    acc_p=ss1/len(shap_all_p)
                                                    print("Shap Accuracy for positive reviews")
                                                    print(acc_p)
                                                    print("Shap only negative reviews")
                                                    for t in shap_exp:
                                                        if t in ann:
                                                            if qrat[t]==0:
                                                                        c=0
                                                                        vb2=0
                                                                        for zz in shap_exp[t]:
                                                                            if zz in ann[t]:
                                                                                if vb2<5:
                                                                                    c=c+1
                                                                                    vb2=vb2+1
                                                                        if len(shap_exp[t])>0:
                                                                            s=float(c)/len(shap_exp[t])
                                                                            if s>0:
                                                                                shap_all_n[t]=s
                                                    ss2=0
                                                    for k in shap_all_n:
                                                        ss2=ss2+float(shap_all_n[k])

                                                    acc_n=ss2/len(shap_all_n)
                                                    print("Shap Accuracy for negative reviews")
                                                    print(acc_n)
                                                    return acc,acc_p,acc_n,shap_exp,shap_values,ann,corpus_train, corpus_test, y_train, y_test,p,qrat,WORDSt
                                    


s_words,stopwords,WORDS,qrat,H11,Rev_text_map=data_processing()
acc,acc_p,acc_n,shap_exp,shap_values,ann,corpus_train,corpus_test,y_train,y_test,p,qrat,WORDSt=Shap_Explanation_Engine(s_words,stopwords,WORDS,qrat,H11,Rev_text_map)



Total Reviews=58112
500 0 500 1000
1000 1000


Liblinear failed to converge, increase the number of iterations.
The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


0.5868973561430794
5469 25720
Shap Accuracy without human-feedback

Shap Accuracy
0.3006432748538001
Shap only positive reviews
Shap Accuracy for positive reviews
0.3297273526824995
Shap only negative reviews
Shap Accuracy for negative reviews
0.24293193717277517


In [2]:
def cluster_feed(KK,qrat,WORDSt,shap_exp,Rev_text_map):
            #Sentance generation
            sent=[]
            sent1=[]
            sent_map=defaultdict(list)
            for ty in WORDSt:
                gh=[]
                gh.append(str(ty))
                #gh1=[]
                #gh2=[]
                for j in WORDSt[ty]:

                    j1=str(j)
                    #gh.append(str(ty))
                    if j1 not in gh:
                        gh.append(j1)

                    #print(gh)


                if gh not in sent:
                        sent.append(gh)


            documents=[]
            #documents1=[]
            for t in sent:
                for jh in t:
                    documents.append(jh)

            for w in sent:
                pass#print(w)
            #K-Means Run 14
            #cluster generation with k-means
            model = Word2Vec(sent, min_count=1)
            X = model[model.wv.vocab]
            NUM_CLUSTERS=KK
            kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,repeats=25)
            assigned_clusters1 = kclusterer.cluster(X,assign_clusters=True)
            #print (assigned_clusters)
            cluster={}
            words = list(model.wv.vocab)
            for i, word in enumerate(words):
              gh=[] 
              gh1=[] 
              gh2=[] 
              if word.isdigit(): 
                cluster[word]=assigned_clusters1[i]
                #print (word + ":" + str(assigned_clusters[i]))
            cluster_final={}
            for j in range(NUM_CLUSTERS):
                gg=[]
                for tt in cluster:
                    if int(cluster[tt])==int(j):
                        if tt not in gg:
                            gg.append(tt)
                if len(gg)>0:
                            cluster_final[j]=gg
            cc=0
            final_clu={}
            lmm=(KK*2)+5
            for t in cluster_final:
                ghh=[]
                vx=0
                for k in cluster_final[t]:
                    if int(k) in WORDS and int(k) in shap_exp or str(k) in shap_exp:
                        if vx<lmm:
                                ghh.append(int(k))
                                vx=vx+1
                if len(ghh)>=2:
                        final_clu[cc]=ghh
                        cc=cc+1
            for k in final_clu:
                pass#print(k,final_clu[k],len(final_clu[k]))
            return final_clu
           

        

                            
                            
                            
cl={}
for kk in range(15,56,10):
        final_clu=cluster_feed(kk,qrat,WORDSt,shap_exp,Rev_text_map)
        cl[kk]=final_clu

C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [3]:
def feedback_form(KK,WORDSt,qrat,final_clu,shap_exp):
        hd=['Review_ID','Truly_Predicted_Class','Original_Text','Explanation','Label']
        with open('feedback_shap_'+str(KK)+'.csv', 'w',newline='') as f:
                writer = csv.writer(f)
                writer.writerow(hd)
                for k in final_clu:
                        #c=-1
                        #c=c+1
                        ghh=[]
                        ghh1=[]
                        c=0
                        c1=0
                        k111=0
                        k222=0
                        try:
                                for t in final_clu[k]:
                                    if qrat[t]==2:
                                        if c<2:
                                            k111=int(t)
                                            c=c+1
                                    elif qrat[t]==0:
                                        if c1<3:
                                            k222=int(t)
                                            pass#print(k,t)
                                            c1=c1+1

                                ghh.append(k111)
                                ghh1.append(k222)
                                if qrat[k111]==2:
                                    ghh.append("Positive")
                                else:
                                    ghh.append("Negative")
                                if qrat[k222]==2:
                                    ghh1.append("Positive")
                                else:
                                    ghh1.append("Negative")
                                s=''
                                for xz1 in WORDSt[k111]:
                                    s=s+xz1+" "
                                ghh.append(s)
                                s1=''
                                for xz1 in WORDSt[k222]:
                                    s1=s1+xz1+" "
                                ghh1.append(s1)
                                #Exp
                                s2=''
                                for xz1 in shap_exp[k111]:
                                    s2=s2+xz1+" "
                                ghh.append(s2)
                                s3=''
                                for xz1 in shap_exp[k222]:
                                    s3=s3+xz1+" "
                                ghh1.append(s3)
                                writer.writerow(ghh)
                                writer.writerow(ghh1)
                        except:
                                continue 
                        
                        
                        
for k in cl:
    feedback_form(k,WORDSt,qrat,cl[k],shap_exp)

                        



In [13]:
import pandas as pd

import shap
def shapp_all_acc():
                def feedback_accuracy(WORDS22,qrat,ann):
                        train_r=[]
                        targets_r=[]
                        m_tid_tr1={}
                        wr=[]
                        tr=[]
                        wr1=[]
                        tr1=[]
                        c=0
                        c1=0
                        tw_wm={}
                        for t in WORDS22:
                            s=''
                            vb=[]
                            #if twit_count[t]==1:
                            for tt in WORDS22[t]:
                                    s=s+str(tt)+" "
                                    train_r.append(tt)
                                    targets_r.append(qrat[t])
                            vb.append(s)
                            wr.append(s)
                            wr1.append(vb)
                            tr.append(qrat[t])
                            tw_wm[t]=s
                        unique_words=[]
                        ss=set( train_r)
                        for w1 in ss:
                                if w1 not in unique_words:
                                    unique_words.append(w1)
                        #Shap

                        corpus_train, corpus_test, y_train, y_test = train_test_split(train_r,targets_r, test_size=0.5, random_state=7)
                        vectorizer = TfidfVectorizer(min_df=1)
                        X_train = vectorizer.fit_transform(corpus_train)
                        X_test = vectorizer.transform(corpus_test)
                        model =svm.LinearSVC(C=100) #sklearn.linear_model.LogisticRegression(penalty="l1", C=0.1)
                        #KNeighborsClassifier(n_neighbors=5)
                        #RandomForestClassifier(max_depth=2, random_state=1)
                        #svm.LinearSVC(C=100) #sklearn.linear_model.LogisticRegression(penalty="l1", C=0.1)
                        model.fit(X_train,y_train)
                        p = model.predict(X_test)
                        prr={}
                        for jj in range(0,len(corpus_test)):
                            prr[corpus_test[jj]]=int(p[jj])
                        print(f1_score(y_test,p,average='micro'))
                        explainer =shap.LinearExplainer(model, X_train, feature_dependence="independent")
                        shap_values = explainer.shap_values(X_train)
                        X_test_array = X_test.toarray() # we need to pass a dense version for the plotting functions
                        feature_names=vectorizer.get_feature_names()
                        print(len(feature_names),len(shap_values))
                        #shap.summary_plot(shap_values, X_test_array, feature_names=vectorizer.get_feature_names())
                        shape_w={}
                        fr={}
                        feature_sh_v=[]
                        for jj in range(0,len(corpus_train)):
                              if abs(sum(shap_values[jj]))>0.4:
                                                      m=abs(sum(shap_values[jj]))
                                                      if corpus_train[jj] not in fr:
                                                                      fr[corpus_train[jj]]=abs(sum(shap_values[jj]))
                                                      elif corpus_train[jj]  in fr:
                                                            if m>fr[corpus_train[jj]]:
                                                                fr[corpus_train[jj]]=m
                        dd1=sorted(fr.items(), key=operator.itemgetter(1),reverse=True)
                        for tt in dd1:
                               feature_sh_v.append(tt[0])
                        '''
                        feature_sh_v1=[]
                        for v in feature_sh_v:
                            n=v.split()
                            for k in n:
                                if k not in feature_sh_v1:
                                    feature_sh_v1.append(k)
                        '''
                        #shap explanations

                        shap_exp={}
                        for t in WORDS22:
                            gh=[]
                            c=0
                            for k in WORDS22[t]:
                                if k in prr:
                                    if qrat[t]==prr[k]:
                                        if k in feature_sh_v:
                                            if k not in gh:
                                                if c<5:
                                                    gh.append(k)
                                                    c=c+1
                            if len(gh)>0:
                                    shap_exp[t]=gh

                        for tt in shap_exp:
                             pass#print(tt,shap_exp[tt])
                        shap_all={}
                        shap_all_p={}
                        shap_all_n={}

                        for t in shap_exp:
                            vc=0
                            if t in ann:
                                c=0
                                for zz in shap_exp[t]:
                                    if zz in ann[t]:
                                        if vc<5:
                                            c=c+1
                                            vc=vc+1
                                if len(shap_exp[t])>0:
                                    s=float(c)/len(shap_exp[t])
                                    if s>0:
                                        shap_all[t]=s
                        ss=0
                        for k in shap_all:
                            ss=ss+float(shap_all[k])

                        acc=ss/len(shap_all)
                        #print("Shap Accuracy without human-feedback"+"\n")
                        #print("Shap Accuracy")
                        #print(acc)
                        #print("Shap only positive reviews")
                        for t in shap_exp:
                            if t in ann:
                                if qrat[t]==2:
                                            c=0
                                            vc1=0
                                            for zz in shap_exp[t]:
                                                if zz in ann[t]:
                                                    if vc1<5:
                                                        c=c+1
                                                        vc1=vc1+1
                                            if len(shap_exp[t])>0:
                                                s=float(c)/len(shap_exp[t])
                                                if s>0:
                                                    shap_all_p[t]=s
                        ss1=0
                        for k in shap_all_p:
                            ss1=ss1+float(shap_all_p[k])
                        if len(shap_all_p)>0:
                            acc_p=ss1/len(shap_all_p)
                        else:
                            acc_p=0
                        #print("Shap Accuracy for positive reviews")
                        #print(acc_p)
                        #print("Shap only negative reviews")
                        for t in shap_exp:
                            if t in ann:
                                if qrat[t]==0:
                                            c=0
                                            vc2=0
                                            for zz in shap_exp[t]:
                                                if zz in ann[t]:
                                                    if vc2<5:
                                                        c=c+1
                                                        vc2=vc2+1
                                            if len(shap_exp[t])>0:
                                                s=float(c)/len(shap_exp[t])
                                                if s>0:
                                                    shap_all_n[t]=s
                        ss2=0
                        for k in shap_all_n:
                            ss2=ss2+float(shap_all_n[k])
                        if len(shap_all_n)>0:
                            acc_n=ss2/len(shap_all_n)
                        else:
                            acc_n=0
                        #print("Shap Accuracy for negative reviews")
                        #print(acc_n)
                        return acc,acc_p,acc_n







                def feed(mn,qrat,cl,w3):
                                sent=[]
                                sent1=[]
                                w33={}
                                cp=0
                                cn=0
                                for kkk in w3:
                                    gvv=[]
                                    if qrat[kkk]==2:
                                        if cp<250:
                                            w33[kkk]=w3[kkk]
                                            cp=cp+1
                                    elif qrat[kkk]==0:
                                        if cn<250:
                                            w33[kkk]=w3[kkk]
                                            cn=cn+1
                                            
                                            
                                sent_map=defaultdict(list)
                                for ty in w3:
                                    gh=[]
                                    gh.append(str(ty))
                                    #gh1=[]
                                    #gh2=[]
                                    for j in w3[ty]:

                                        j1=str(j)
                                        #gh.append(str(ty))
                                        if j1 not in gh:
                                            gh.append(j1)

                                    if gh not in sent:
                                            sent.append(gh)
                                documents=[]
                                #documents1=[]
                                for t in sent:
                                    for jh in t:
                                        documents.append(jh)
                                hh="feedback_shap_"+str(mn)+".csv"
                                ps={}
                                ns={}
                                vot={}
                                vtt={}
                                f1=pd.read_csv(hh)
                                vot={}
                                vtt={}
                                ll=0
                                for col in f1.columns:
                                    if 'Label' in col:
                                        ll=ll+1
                                m=len(f1['Review_ID'])
                                for t in range(0,m):
                                    #print(f1['Review_ID'][t])
                                    vtt[f1['Review_ID'][t]]=f1['Explanation'][t]
                                    gh=[]
                                    for vv in range(1,ll+1):
                                                  vb1="Label"+str(vv)
                                                  #print(f1[vb1][t])
                                                  gh.append(f1[vb1][t])
                                    vot[f1['Review_ID'][t]]=gh


                                for uy in vot:
                                    cp=0
                                    cn=0
                                    for kk in vot[uy]:
                                        if int(kk)==1:
                                            cp=cp+1
                                        else:
                                            cn=cn+1
                                    if cp>=cn:
                                        ps[uy]=vtt[uy]
                                    elif cp<cn:
                                        ns[uy]=vtt[uy]
                    
                                cl1=cl[mn]

                                WORDS23={}
                                rm=[]
                                for jj in ns:
                                    for kj in ns[jj]:
                                        rm.append(kj)

                                model = Word2Vec(sent, min_count=1)
                                rme={}
                                for uu in ns:
                                    for k in cl1:
                                        if uu in cl1[k]:
                                            for kk in cl1[k]:
                                                gg=[]
                                                zz=0
                                                if kk in w33:
                                                    for vv in w33[kk]:
                                                        if vv in ns[uu]:
                                                            continue
                                                        else:
                                                            if zz<5:
                                                                gg.append(vv)
                                                                zz=zz+1
                                                    rme[kk]=gg


                                for t in cl1:
                                    for k in ps:
                                        if k in cl1[t]:
                                            for kk in cl1[t]:
                                                chu1=[]
                                                vb={}
                                                for v in ps[k]:
                                                    vb1={}
                                                    if kk in w33:
                                                        for v1 in w33[kk]:
                                                            try:
                                                                    gh1=model.similarity(v,v1)
                                                                    if gh1>0.4:
                                                                                  vb1[v1]=float(gh1)
                                                            except:
                                                                continue 
                                                        for jk in vb1:
                                                                        if jk in vb:
                                                                            if float(vb1[jk])>=float(vb[jk]):
                                                                                #print(jk,vb1[jk],vb[jk])
                                                                                vb[jk]=vb1[jk]
                                                                        else:
                                                                            vb[jk]=vb1[jk]

                                                dd1=sorted(vb.items(), key=operator.itemgetter(1),reverse=True)
                                                cc=0
                                                for kkk in dd1:
                                                    if kkk[0] not in chu1:
                                                        if cc<5:
                                                                chu1.append(kkk[0])
                                                                cc=cc+1
                                                if len(chu1)>0 :
                                                    if kk in w33:
                                                        WORDS23[kk]=chu1 
                                WORDS25={}
                                for t in WORDS23:
                                    cc=0
                                    vcx=[]
                                    if t not in rme:
                                        WORDS25[t]=WORDS23[t]
                                    elif t in rme:
                                        vcc=WORDS23[t]+rme[t]
                                        for zz in vcc:
                                            if cc<5:
                                                vcx.append(zz)
                                                cc=cc+1
                                        WORDS25[t]=vcx
                                print(len(WORDS25))
                                for cc in rme:
                                    fg=[]
                                    vc4=0
                                    if cc not in WORDS25:
                                        for bb in rme[cc]:
                                            if vc4<5:
                                                fg.append(bb)
                                                vc4=vc4+1
                                        WORDS25[cc]=fg
                                print(len(WORDS25))

                                return WORDS25






                all_a={}
                on_p={}
                on_n={}
                for mn in range(15,56,10):     
                        WORDS25=feed(mn,qrat,cl,WORDSt)
                        acc,acc_p,acc_n=feedback_accuracy(WORDS25,qrat,ann)  
                        all_a[mn]=acc
                        on_p[mn]=acc_p
                        on_n[mn]=acc_n


                for tt in all_a:
                    print(tt,all_a[tt],on_p[tt],on_n[tt])
                    print("\n\n")

                    
shapp_all_acc()



C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


0
235
0.576530612244898
423 587


Liblinear failed to converge, increase the number of iterations.
The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)
C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


0
167
0.49760765550239233
320 417


Liblinear failed to converge, increase the number of iterations.
The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)
C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


...

Liblinear failed to converge, increase the number of iterations.
The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)
C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


0
93
0.5064377682403434
199 232


Liblinear failed to converge, increase the number of iterations.
The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)
C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


0
226
0.5008849557522124
403 565
15 0.7416666666666667 0.7467948717948718 0.7083333333333333



25 0.746031746031746 0.788888888888889 0.6388888888888888



35 0.7738095238095238 0.8333333333333334 0.625



45 0.7727272727272727 0.8 0.75



55 0.7777777777777778 0.7745098039215685 0.7857142857142857





Liblinear failed to converge, increase the number of iterations.
The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


In [181]:
 with open('standard_Result.csv', 'w',newline='') as f:
                writer = csv.writer(f)
                hd=['id','standard exp']
                writer.writerow(hd)
                for k in ann:
                    gh=[]
                    gh.append(k)
                    s=''
                    for bb in ann[k]:
                        s=s+str(bb)+" "
                    gh.append(s)
                    writer.writerow(gh)

In [182]:
with open('Original_Evid.csv', 'w',newline='') as f:
                writer = csv.writer(f)
                hd=['rid','word_evid']
                writer.writerow(hd)
                for k in WORDSt:
                    gh=[]
                    gh.append(k)
                    s=''
                    for bb in WORDSt[k]:
                        s=s+str(bb)+" "
                    gh.append(s)
                    writer.writerow(gh)

In [183]:
with open('true_rating.csv', 'w',newline='') as f:
                writer = csv.writer(f)
                hd=['rid','rating']
                writer.writerow(hd)
                for k in qrat:
                    gh=[]
                    gh.append(k)
                    #s=''
                    #for bb in WORDSt[k]:
                       # s=s+str(bb)+" "
                    gh.append(qrat[k])
                    writer.writerow(gh)

In [188]:
with open('cluster_25.csv', 'w',newline='') as f:
                writer = csv.writer(f)
                hd=['cid','queries']
                writer.writerow(hd)
                for k in cl[25]:
                    gh=[]
                    gh.append(k)
                    s=''
                    for bb in cl[25][k]:
                        s=s+str(bb)+" "
                    gh.append(s)
                    writer.writerow(gh)